In [2]:
from bs4 import BeautifulSoup
import lxml #is a parser for HTMl
import webscraping
import requests
import os
import re
import csv
#from selenium import webdriver
from langdetect import detect

We shoul generalize these functions... in webscraping.py!

## **[RQ1.1] Get the list of books**

We start to scrape the three pages to extract the URLs for each of them

In [32]:
UrlsFiles = open("urlpages.txt", "w")

url = "https://www.goodreads.com/list/show/1.Best_Books_Ever?page="+str(1)
page = requests.get(url)
soup = BeautifulSoup(page.content, features='lxml')
for a in soup.find_all('a', class_="bookTitle"): #.contents[0].split('\n')[1].strip()
    UrlsFiles.write(a.get('href')+'\n')

#UrlsFiles.truncate(UrlsFiles.tell()-1) see how to remove the '\n'
UrlsFiles.close()

## **RQ[1.2] Crawl books**

In [10]:
headpart = "https://www.goodreads.com"

In [11]:
direct = "htmlpages" #Directory  
parent_dir = "./" #Parent Directory path 
pathAncestor = os.path.join(parent_dir, direct) #Path
os.mkdir(pathAncestor) #create the folder in the path

In [12]:
for i in range(1,101):
    os.makedirs(os.path.join(pathAncestor, 'page ' + str(i)))

In [13]:
UrlsFiles = open("urlpages.txt", "r")

for i, x in enumerate(UrlsFiles,1):
    subdirectory = pathAncestor + "/page " + str(i)
    article_name = "/article_"+str(i)+".html"
    
    complete_path = subdirectory + article_name
    with open(complete_path, "wb") as ip_file:
        link = headpart + x
        page = requests.get(link)
        
        soup = BeautifulSoup(page.text, features='lxml')
        
        ip_file.write(soup.encode('utf-8'))
        ip_file.close()

UrlsFiles.close()

## **RQ[1.3] Parse downloaded pages**

In [44]:
def scrap_book(href, tsv_writer, Url):
    r = requests.get(href) #call to have the html page with the useful information for my analysis
    soup = BeautifulSoup(r.text, features="lxml") #parse the text
    
    #extract rating and review count
    ratings = soup.find_all('a', href="#other_reviews") #search the ratings in its
    rating_count = -1
    rating = -1
    for raiting in ratings:
        if raiting.find_all('meta', itemprop="ratingCount"):
            ratingCount = raiting.text.replace('\n', '').strip().split(' ')[0].replace(',', '')
        elif raiting.find_all('meta', itemprop="reviewCount"):
            reviewCount = raiting.text.replace('\n', '').strip().split(' ')[0].replace(',', '')
    
    #extract the book title
    bookTitle = soup.find_all('h1')[0].contents[0].replace('\n', '').strip()

    #extract the book authors
    bookAuthors = soup.find_all('span', itemprop='name')[0].contents[0]
    
    #extract the book authors, we shoul FIX it.
    try:
        Plot = soup.find_all('div', id="description")[0].contents[3].text
    except:
        Plot = soup.find_all('div', id="description")[0].contents[1].text
    if detect(Plot) != "en":
        Plot = "NaN"
    
    #extract the date
    date = soup.find_all('div', id="details")[0].contents[3].text.replace('\n', '').strip().split()
    Published = date[1]+" "+date[2]+" "+date[3]
    
    #Rating Value
    ratingValue = soup.find('span', itemprop="ratingValue").text.strip()
    
    #Number of pages
    NumberofPages = soup.find('span', itemprop="numberOfPages").text.split()[0]
    
    #Title series
    bookSeries = soup.find_all('a', href= re.compile(r'/series/*'))
    if not bookSeries:
        bookSeries = "NaN"
    else:
        bookSeries = soup.find_all('a', href= re.compile(r'/series/*'))[0].contents[0].strip()
    
    #Places
    Setting = []
    for places in soup.find_all('a', href= re.compile(r'/places/*')):
        Setting.append(places.text)
    Setting = ", ".join(Setting) if len(Setting)>=1 else "NaN"
        
    #list of characters
    Characters = []
    for character in soup.find_all('a', href= re.compile(r'/characters/*') ):
        Characters.append(character.text)
    Characters = ", ".join(Characters) if len(Characters)>=1 else "NaN"
    
    tsv_writer.writerow([bookTitle, bookSeries, bookAuthors, ratingValue, ratingCount, reviewCount, Plot, NumberofPages, Published, Characters, Setting, Url])

In [45]:
UrlsFiles = open("urlpages.txt", "r")

with open('./output.tsv', 'w', encoding="utf-8", newline='') as out_file:
    tsv_writer = csv.writer(out_file, delimiter='\t')
    tsv_writer.writerow(['bookTitle', 'bookSeries', 'bookAuthors', 'ratingValue', 
                    'ratingCount', 'reviewCount', 'Plot', 'NumberofPages', 'Published',
                    'Characters', 'Setting', 'Url'])
    for link in UrlsFiles:
        scrap_book("https://www.goodreads.com" + link, tsv_writer, link)

UrlsFiles.close()

In [43]:
UrlsFiles = open("urlpages.txt", "r")

for link in UrlsFiles:
    r = requests.get("https://www.goodreads.com" + link) #call to have the html page with the useful information for my analysis
    soup = BeautifulSoup(r.text, features="lxml") #parse the text
    
    try:
        Plot = soup.find_all('div', id="description")[0].contents[3].text
    except:
        Plot = soup.find_all('div', id="description")[0].contents[1].text
    print(Plot)

UrlsFiles.close()
#extract the book authors, we shoul FIX it.
#Plot = ''.join([c for c in soup.find_all('div', id="description")[0].text if c != '</br>'])
#Plot = Plot.replace("...more", "").strip()
#if detect(Plot) != "en":
#    Plot = "NaN"
#print(Plot)

Could you survive on your own in the wild, with every one out to make sure you don't live to see the morning?In the ruins of a place once known as North America lies the nation of Panem, a shining Capitol surrounded by twelve outlying districts. The Capitol is harsh and cruel and keeps the districts in line by forcing them all to send one boy and one girl between the ages of twelve and eighteen to participate in the annual Hunger Games, a fight to the death on live TV.Sixteen-year-old Katniss Everdeen, who lives alone with her mother and younger sister, regards it as a death sentence when she steps forward to take her sister's place in the Games. But Katniss has been close to dead before—and survival, for her, is second nature. Without really meaning to, she becomes a contender. But if she is to win, she will have to start making choices that weight survival against humanity and life against love.
There is a door at the end of a silent corridor. And it’s haunting Harry Pottter’s dreams

KeyboardInterrupt: 